In [1]:
#이 부분은 커널 시작할때 한번만 실행

import csv
import youtube_dl
import os
import cv2
import numpy as np

def sec_to_time(sec):
    time = ""
    temp = 0
    
    for i in range(3):
        temp = sec % 60
        if temp < 10:
            time = "0" + str(temp) + time
        else:
            time = str(sec % 60) + time
        sec = sec // 60
        
        if i < 2:
            time = ":" + time
    
    return time



In [100]:
#작업하는 영상 바뀔때만 입력 정보 바꿔서 실행
#실행할때마다 이름이 vid.mp4, audio.wav인 파일을 삭제하니 주의

# 손으로 입력해줄 정보들

vidlink = 'https://www.youtube.com/watch?v=yvTsAR0vI9g';
start_sec = 4
end_sec = 48

# start & end sec 기준으로 잘라진 영상 다운로드, 영상에서 오디오 추출

ydl = youtube_dl.YoutubeDL({'format': '18'}, )

with ydl:
    video = ydl.extract_info(
        vidlink,
        download=False
    )
url = video['url']
fps = video['fps']

start_time = sec_to_time(start_sec)
duration_time = sec_to_time(end_sec - start_sec)

if os.path.exists('vid.mp4'):
    os.remove('vid.mp4')
if os.path.exists('audio.wav'):
    os.remove('audio.wav')
os.system("ffmpeg -i '%s' -ss %s -t %s -async 1 -strict -2 'vid.mp4'" % (url, start_time, duration_time))
os.system("ffmpeg -i vid.mp4 -ab 160k -ac 2 -ar 44100 -vn audio.wav")

[youtube] yvTsAR0vI9g: Downloading webpage


0

In [102]:
##### padding 수정할때마다 실행, 실행 후 output.mp4 확인하면 됨
##8박자마다 frame 수, 1박자마다 frame 수가 하단에 출력되니 참고해서 padding 작성
##padding이 잘 지정된 것 같으면 slice_frame.csv의 값을 복사해서 google drive의
# 스프레드시트에 복사

bpm = 144
padding = 30

# 8박자에 들어가는 frame 수 계산
frame_per_min = fps*60
frame_per_8beat = frame_per_min * 8/bpm
print("fps: ", fps)
print("8박자에 소요되는 프레임 수: ", frame_per_8beat)
print("1박자에 소요되는 프레임 수: ", frame_per_8beat/8)

#8박자마다 4frame 이미지를 검정색으로 변환
vidcap = cv2.VideoCapture('vid.mp4')

width = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fourcc = cv2.VideoWriter_fourcc('F', 'M', 'P', '4')
out = cv2.VideoWriter('vid2.mp4', fourcc, fps, (int(width), int(height)))

count = 0
slice_frame = padding

start = 0;
end = 0;

flag = False
frame = []
with open('slice_frame.csv', 'w', newline='') as f:
    makewrite = csv.writer(f)
    while(vidcap.isOpened()):
        ret, image = vidcap.read()
        count += 1

        if ret == False:
                break

        if(count >= (int(slice_frame)) and count <= (int(slice_frame)+1)):
            if(count == int(slice_frame)+1):
                start = end
                end = count
                frame.append(str(start-20)+'~'+str(end+20))
            black = np.zeros((int(height), int(width), 3), np.uint8)
            out.write(black)
            flag = True
        else:
            if flag == True:
                slice_frame += frame_per_8beat
                flag = False
            out.write(image)

    out.release()
    vidcap.release()
    makewrite.writerow(frame)
    
# 편집된 영상과 소리 합치기
if os.path.exists('output.mp4'):
    os.remove('output.mp4')
os.system("ffmpeg -i vid2.mp4 -i audio.wav -c:v copy -c:a aac output.mp4")
os.remove('vid2.mp4')

fps:  30
8박자에 소요되는 프레임 수:  100.0
1박자에 소요되는 프레임 수:  12.5
